In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


from tensorflow.keras.preprocessing import image_dataset_from_directory

In [15]:
train_dir = "C:/Users/Lukas/Desktop/SuperClassModel/data/fruits-360/Training"
validation_dir = "C:/Users/Lukas/Desktop/SuperClassModel/data/fruits-360/Test"

BATCH_SIZE = 32
IMG_SIZE = (160, 160)



In [21]:
train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Lukas\\.keras\\datasets\\fruits-360\\Test'